In [ ]:
import spacy
!python -m spacy download en_core_web_lg
#!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_trf-3.2.0/en_core_web_trf-3.2.0-py3-none-any.whl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 587.7 MB 16 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:
nlp=spacy.load("en_core_web_lg")

In [ ]:
sentance="Donad Trump was President of USA"

In [ ]:
doc=nlp(sentance)
doc

Donad Trump was President of USA

In [ ]:
type(doc)

spacy.tokens.doc.Doc

In [ ]:
doc.ents

(Donad Trump, USA)

In [ ]:
doc.ents[0],type(doc.ents[0])

(Donad Trump, spacy.tokens.span.Span)

In [ ]:
from spacy import displacy
displacy.render(doc,style="ent",jupyter=True)

In [ ]:
# https://www.kaggle.com/datasets/finalepoch/medical-ner     data set link
import json
with open("/content/sample_data/Corona2.json","r") as file:
  data=json.load(file)

In [ ]:
data['examples'][0].keys()

dict_keys(['id', 'content', 'metadata', 'annotations', 'classifications'])

In [ ]:
data['examples'][0]['content']

"While bismuth compounds (Pepto-Bismol) decreased the number of bowel movements in those with travelers' diarrhea, they do not decrease the length of illness.[91] Anti-motility agents like loperamide are also effective at reducing the number of stools but not the duration of disease.[8] These agents should be used only if bloody diarrhea is not present.[92]\n\nDiosmectite, a natural aluminomagnesium silicate clay, is effective in alleviating symptoms of acute diarrhea in children,[93] and also has some effects in chronic functional diarrhea, radiation-induced diarrhea, and chemotherapy-induced diarrhea.[45] Another absorbent agent used for the treatment of mild diarrhea is kaopectate.\n\nRacecadotril an antisecretory medication may be used to treat diarrhea in children and adults.[86] It has better tolerability than loperamide, as it causes less constipation and flatulence.[94]"

In [ ]:
data['examples'][0]['annotations']

[{'id': '0825a1bf-6a6e-4fa2-be77-8d104701eaed',
  'tag_id': 'c06bd022-6ded-44a5-8d90-f17685bb85a1',
  'end': 371,
  'start': 360,
  'example_id': '18c2f619-f102-452f-ab81-d26f7e283ffe',
  'tag_name': 'Medicine',
  'value': 'Diosmectite',
  'correct': None,
  'human_annotations': [{'timestamp': '2020-03-21T00:24:32.098000Z',
    'annotator_id': 1,
    'tagged_token_id': '0825a1bf-6a6e-4fa2-be77-8d104701eaed',
    'name': 'Ashpat123',
    'reason': 'exploration'}],
  'model_annotations': []},
 {'id': '145f62b1-bbf5-42f1-8ad5-9c7e08337bf0',
  'tag_id': 'c06bd022-6ded-44a5-8d90-f17685bb85a1',
  'end': 408,
  'start': 383,
  'example_id': '18c2f619-f102-452f-ab81-d26f7e283ffe',
  'tag_name': 'Medicine',
  'value': 'aluminomagnesium silicate',
  'correct': None,
  'human_annotations': [{'timestamp': '2020-03-21T00:24:43.692000Z',
    'annotator_id': 1,
    'tagged_token_id': '145f62b1-bbf5-42f1-8ad5-9c7e08337bf0',
    'name': 'Ashpat123',
    'reason': 'exploration'}],
  'model_annotations':

In [ ]:
dataset=[]
for example in data['examples']:
  temp_dict={}
  temp_dict["text"]= example['content']
  temp_dict['entities']=[]
  for annotation in example['annotations']:
    start=annotation['start']
    end=annotation['end']
    label=annotation['tag_name'].upper()
    temp_dict['entities'].append((start,end,label))
  dataset.append(temp_dict)

In [ ]:
dataset

[{'text': "While bismuth compounds (Pepto-Bismol) decreased the number of bowel movements in those with travelers' diarrhea, they do not decrease the length of illness.[91] Anti-motility agents like loperamide are also effective at reducing the number of stools but not the duration of disease.[8] These agents should be used only if bloody diarrhea is not present.[92]\n\nDiosmectite, a natural aluminomagnesium silicate clay, is effective in alleviating symptoms of acute diarrhea in children,[93] and also has some effects in chronic functional diarrhea, radiation-induced diarrhea, and chemotherapy-induced diarrhea.[45] Another absorbent agent used for the treatment of mild diarrhea is kaopectate.\n\nRacecadotril an antisecretory medication may be used to treat diarrhea in children and adults.[86] It has better tolerability than loperamide, as it causes less constipation and flatulence.[94]",
  'entities': [(360, 371, 'MEDICINE'),
   (383, 408, 'MEDICINE'),
   (104, 112, 'MEDICALCONDITION

In [ ]:
dataset[0]['text'][6:23]

'bismuth compounds'

In [ ]:
from spacy.tokens import DocBin
from tqdm import tqdm

In [ ]:
nlp=spacy.blank("en")   # load blank model
doc_bin=DocBin()       # create doc bin object

In [ ]:
from spacy.util import filter_spans
for training_example in tqdm(dataset):
  text=training_example['text']
  labels=training_example['entities']
  doc=nlp.make_doc(text)
  ents=[]
  for start,end,label in labels:
    span=doc.char_span(start,end,label=label,alignment_mode="contract")
    if span is None:
      print("skipping entity")
    else:
      ents.append(span)
  filtered_ents=filter_spans(ents)
  doc_ents=filtered_ents
  doc_bin.add(doc)
doc_bin.to_disk("train.spacy")

100%|██████████| 31/31 [00:00<00:00, 401.98it/s]

skipping entity
skipping entity
skipping entity
skipping entity
skipping entity
skipping entity
skipping entity
skipping entity
skipping entity
skipping entity
skipping entity
skipping entity
skipping entity
skipping entity
skipping entity
skipping entity


In [ ]:
# here config file is select gpu so must run colab on gpu
# https://spacy.io/usage/training#quickstart  config file link
!python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
!python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./train.spacy

✔ Created output directory: output
ℹ Saving to output directory: output
ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0

=========================== Initializing pipeline ===========================
[2022-12-04 08:13:17,937] [INFO] Set up nlp object from config
INFO:spacy:Set up nlp object from config
[2022-12-04 08:13:17,948] [INFO] Pipeline: ['tok2vec', 'ner']
INFO:spacy:Pipeline: ['tok2vec', 'ner']
[2022-12-04 08:13:17,952] [INFO] Created vocabulary
INFO:spacy:Created vocabulary
[2022-12-04 08:13:19,431] [INFO] Added vectors: en_core_web_lg
INFO:spacy:Added vectors: en_core_web_lg
tcmalloc: large alloc 1233977344 bytes == 0x5a0f0000 @  0x7f53f1ee82a4 0x7f53e6828e09 0x7f53e6827cdf 0x7f53e6824675 0x7f53e6824e2e 0x4f7585 0x49ca7c 0x55e571 0x5d7cf1 0x49ced5 0x7f529ddced78 0x7f529ddd170e 0x7f529ddd95a7 0x7f529dddce85 0x5d813c 0x55f3fd 0x5d7c18 0x49ec69 0x7f529ddced78 0x7f529ddd170e 0x7f529ddd8e4b 0x5d746e 0x5d813c 0x560200 0x55e858 0x5d7cf1 0x49ced5 0x4fe993 0x49ced5 0x55e85

In [ ]:
nlp_ner = spacy.load("/content/output/model-best")

In [ ]:
doc = nlp_ner("While bismuth compounds (Pepto-Bismol) decreased the number of bowel movements in those with travelers' diarrhea, they do not decrease the length of illness.[91] Anti-motility agents like loperamide are also effective at reducing the number of stools but not the duration of disease.[8] These agents should be used only if bloody diarrhea is not present.")

#colors = {"PATHOGEN": "#F67DE3", "MEDICINE": "#7DF6D9", "MEDICALCONDITION":"#a6e22d"}
#options = {"colors": colors} 

spacy.displacy.render(doc,style="ent",jupyter=True)

/usr/local/lib/python3.8/dist-packages/spacy/displacy/__init__.py:206: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)
